In [1]:
from __future__ import division, print_function
import numpy as np
from hmmlearn.hmm import MultinomialHMM
import poetrytools
import itertools

In [2]:
%run 'preprocessing.ipynb'

In [3]:
def get_rhymes(words):
    num_words = len(words)
    rhymes = np.empty((num_words, num_words))
    
    for i in range(num_words):
        for j in (k for k in range(num_words) if not k == i):
            if poetrytools.rhymes(words[i], words[j]):
                rhymes[i][j] = 1
    return rhymes

In [4]:
quatrain_rhymes = get_rhymes(quatrain_words)
volta_rhymes = get_rhymes(volta_words)
couplet_rhymes = get_rhymes(couplet_words)

In [13]:
reversed_quatrain_model = MultinomialHMM(n_components=30, n_iter=100, verbose=True)
reversed_volta_model = MultinomialHMM(n_components=30, n_iter=100, verbose=True)
reversed_couplet_model = MultinomialHMM(n_components=30, n_iter=100, verbose=True)
reversed_quatrain_model.fit(np.concatenate(reversed_converted_quatrain_lines), quatrain_lengths)
reversed_volta_model.fit(np.concatenate(reversed_converted_volta_lines), volta_lengths)
reversed_couplet_model.fit(np.concatenate(reversed_converted_couplet_lines), couplet_lengths)

         1      -76015.6398             +nan
         2      -62170.1170      +13845.5227
         3      -62164.9582          +5.1588
         4      -62148.9640         +15.9942
         5      -62098.6633         +50.3008
         6      -61949.7554        +148.9078
         7      -61595.9441        +353.8113
         8      -61046.5080        +549.4361
         9      -60509.3053        +537.2027
        10      -60141.7790        +367.5262
        11      -59954.5267        +187.2523
        12      -59860.3279         +94.1988
        13      -59787.9181         +72.4098
        14      -59691.5676         +96.3505
        15      -59536.3465        +155.2211
        16      -59317.5607        +218.7858
        17      -59070.2556        +247.3051
        18      -58833.4760        +236.7797
        19      -58613.5523        +219.9237
        20      -58379.9268        +233.6254
        21      -58097.3581        +282.5688
        22      -57757.0455        +340.3126
        23

MultinomialHMM(algorithm='viterbi', init_params='ste', n_components=30,
        n_iter=100, params='ste',
        random_state=<mtrand.RandomState object at 0x111e745a0>,
        startprob_prior=1.0, tol=0.01, transmat_prior=1.0, verbose=True)

In [18]:
def generate_line(model, words, seed=None, rhymes=None):
    syllables_left = 10
    last_stress = 0
    
    line= []
    start_probs = model.startprob_
    emission_probs = model.emissionprob_
    transition_probs = model.transmat_
    
    start_state = np.random.choice(len(start_probs), p = start_probs)
    
    if seed is not None:
        possible_start_emissions = np.where(rhymes[seed] == 1)
        probs = np.array(emission_probs[start_state][possible_start_emissions])

        scaled_probs = probs / sum(probs)
        while True:
            start_emission = np.random.choice(possible_start_emissions[0], p=scaled_probs)
            start_stress = poetrytools.stress(words[start_emission])
            if len(start_stress) == 1 or int(start_stress[-1]) == 1 :
                break
    else:
        while True:
            start_emission = np.random.choice(len(emission_probs[start_state]), p=emission_probs[start_state])
            start_stress = poetrytools.stress(words[start_emission])
            if len(start_stress) == 1 or int(start_stress[-1]) == 1:
                break
    
    line.append(start_emission)
    start_stress = poetrytools.stress(words[start_emission])
    syllables_left -= len(start_stress)
    
    if len(start_stress) == 1:
        prev_starting_stress = 1
    else:
        prev_starting_stress = int(start_stress[0])

    curr_state = start_state
    while syllables_left > 0:
        possible_transitions = transition_probs[curr_state]
        curr_state = np.random.choice(len(possible_transitions), p=possible_transitions)
        possible_emissions = emission_probs[curr_state]
        while True:
            curr_emission = np.random.choice(len(possible_emissions), p=possible_emissions)
            curr_stress = poetrytools.stress(words[curr_emission])
            if len(curr_stress) == 1:
                prev_starting_stress = 1 - prev_starting_stress
                syllables_left -= 1
                break
            elif len(curr_stress) > syllables_left or int(curr_stress[-1]) == prev_starting_stress:
                continue
            else:
                prev_starting_stress = int(curr_stress[0])
                syllables_left -= len(curr_stress)
                break
        line.append(curr_emission)

    return line

def convert_line(sample, words):
    ret = ''
    i = 0
    for word in reversed(sample):
        curr_word = words[word]
        if i == 0 or len(curr_word) == 1:
            ret += curr_word.title() + ' '
        else:
            ret += curr_word + ' '
        i += 1
    return ret

def generate_pair(model, words, rhymes):
    while True:
        a_line = generate_line(model, words)
        seed = a_line[0]
        if len(np.where(rhymes[seed] == 1)[0]) > 0:
            b_line = generate_line(model, words, seed, rhymes)
            return a_line, b_line
        
def generate_rhyming_and_meter_sonnet():
    sonnet = ''
    a_lines = []
    b_lines = []
    
    for _ in range(4):
        a_line, b_line = generate_pair(reversed_quatrain_model, quatrain_words, quatrain_rhymes)
        a_lines.append(a_line)
        b_lines.append(b_line)
    
    for i in range(2):
        sonnet += convert_line(a_lines[2 * i], quatrain_words) + '\n'
        sonnet += convert_line(a_lines[2 * i + 1], quatrain_words) + '\n'
        sonnet += convert_line(b_lines[2 * i], quatrain_words) + '\n'
        sonnet += convert_line(b_lines[2 * i + 1], quatrain_words) + '\n'
    
    a_lines = []
    b_lines = []
    
    for _ in range(2):
        a_line, b_line = generate_pair(reversed_volta_model, volta_words, volta_rhymes)
        a_lines.append(a_line)
        b_lines.append(b_line)
    
    sonnet += convert_line(a_lines[0], volta_words) + '\n'
    sonnet += convert_line(a_lines[1], volta_words) + '\n'
    sonnet += convert_line(b_lines[0], volta_words) + '\n'
    sonnet += convert_line(b_lines[1], volta_words) + '\n'
    
    a_line, b_line = generate_pair(reversed_couplet_model, couplet_words, couplet_rhymes)
    sonnet += convert_line(a_line, couplet_words) + '\n'
    sonnet += convert_line(b_line, couplet_words) + '\n'
    
    return sonnet

def generate_10_rhyming_and_meter_sonnets():
    sonnets = ''
    for i in range(10):
        print('Generating Sonnet' + str(i + 1))
        sonnets += str(i) + '\n' + generate_rhyming_and_meter_sonnet() + '\n'
    
    f = open("project2data/rhyming_and_meter_shakespeare.txt","w")
    f.write(sonnets)
    return sonnets

In [19]:
print(generate_10_rhyming_and_meter_sonnets())

Generating Sonnet1
Generating Sonnet2
Generating Sonnet3
Generating Sonnet4
Generating Sonnet5
Generating Sonnet6
Generating Sonnet7
Generating Sonnet8
Generating Sonnet9
Generating Sonnet10
0
You is to time not my thinks youthful rose 
Not so by to heart with where thou by know 
But mine mightier give of most foes shows 
Excuse where on root crow my to the no 
Old it and for my bonds when line you use 
Sweet now of youth's in great his reason worth 
Which see with my love to thy now to muse 
Confine how sigh then space is the old dearth 
Thou no am loves but my those from thou face 
Good the pursuing of A winter's stained 
Should the had should policy A both race 
Within his world to side thine scope confined 
We 'thou with sweet says hast have be mayst dead 
Poor an not this to love the in thy read 

1
Swears which forfeit thee by summers' forsworn 
Penury now the self deep gilded light 
Men'S one that tell such since by dark hand torn 
Truth and that with my course own former might 

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:16: RuntimeWarning: invalid value encountered in true_divide
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:18: RuntimeWarning: invalid value encountered in less


In [141]:
# checks if line is in iambic pentameter (i.e. 0101010101 stress pattern)
def check_iambic_pentameter(line):
    # get the stresses from cmu dict 
    # if word is 1 syllable, then have the option for it to be stressed or unstressed
    stresses = []
    for i in line.split(' '):
        stress = poetrytools.stress(i)
        if len(stress) == 1:
            stresses.append(['0','1'])
        else:
            stresses.append([stress])
    
    # make combination of all possible stress patterns
    result = [[]]
    final = []
    for pool in stresses:
        result = [x+[y] for x in result for y in pool]
    final = [''.join(i) for i in result]
    
    # return if any pattern fits iambic pentameter 
    return ('0101010101' in final)